In [ ]:
import tensorflow as tf
%reload_ext autoreload
%autoreload 2

In [ ]:

from src.stages.split import split_data_from_config
# Splitting data into training and validation sets
split_data_from_config('./params.yaml');

In [ ]:
from src.stages.datasets import create_datasets_from_config
create_datasets_from_config('./params.yaml');

In [ ]:
from src.stages.model_setup import create_model_from_config
create_model_from_config('./params.yaml')
model = tf.keras.models.load_model('./models/untrained_model.h5')

In [ ]:
from src.stages.training import train_from_config
history, trained_model = train_from_config('./params.yaml')

In [ ]:
from src.stages.predict import predict_from_config
predict_from_config('./params.yaml')